# OBIA Feature Extraction with Sentinel-2 Imagery


This notebook performs object-based feature extraction from Sentinel-2 imagery using the `nickyspatial` library.

Steps:
1. Load Sentinel-2 bands (Red and NIR)
2. Compute NDVI
3. Segment the image using SLIC
4. Convert segments to vector objects
5. Extract object statistics (NDVI, area, shape metrics)
6. Compute global texture (GLCM contrast)
7. Save and visualize the results
    

In [ ]:

import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops

from nickyspatial import SlicSegmentation, attach_basic_stats, attach_shape_metrics, layer_to_vector


## Load Sentinel-2 Red and NIR bands from multiband TIFF

In [ ]:

multiband_path = "sentinel2.tif"

with rasterio.open(multiband_path) as src:
    print(f"Number of bands: {src.count}")
    red = src.read(3).astype("float32")  # Adjust if band 3 is Red
    nir = src.read(4).astype("float32")  # Adjust if band 4 is NIR
    transform = src.transform
    profile = src.profile


## Compute NDVI

In [ ]:

ndvi = (nir - red) / (nir + red)
ndvi = np.clip(ndvi, -1, 1)

plt.imshow(ndvi, cmap='RdYlGn')
plt.title("NDVI")
plt.colorbar()
plt.show()


## Apply SLIC Segmentation using nickyspatial

In [ ]:

slic = SlicSegmentation()
segments = slic.fit(ndvi)

plt.imshow(segments, cmap='tab20')
plt.title("Segmented Image")
plt.colorbar()
plt.show()


## Convert Segments to Vector and Attach Statistics

In [ ]:

layer = {'raster': ndvi, 'segments': segments, 'transform': transform}
gdf = layer_to_vector(layer)
gdf = attach_basic_stats(gdf, ndvi, stat_name='mean_ndvi')
gdf = attach_shape_metrics(gdf)


## Compute Global GLCM Contrast (Optional)

In [ ]:

ndvi_int = ((ndvi + 1) * 127.5).astype('uint8')
glcm = greycomatrix(ndvi_int, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
contrast = greycoprops(glcm, 'contrast')[0, 0]
gdf['glcm_contrast'] = contrast


## Save to GeoJSON

In [ ]:

gdf.to_file("obia_features.geojson", driver="GeoJSON")


## Visualize NDVI per Object

In [ ]:

gdf.plot(column='mean_ndvi', cmap='YlGn', legend=True)
plt.title("Mean NDVI per Object")
plt.show()
